In [15]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import check_classification_targets
from sklearn.preprocessing import normalize
from joblib import Parallel, delayed
from scipy import linalg

class PGMHQC_fast(BaseEstimator, ClassifierMixin):
    """The Pretty Good Measurement (PGM) - Helstrom Quantum Centroid (HQC) classifier is a 
    quantum-inspired supervised classification approach for data with multiple classes.
                         
    Parameters
    ----------
    rescale : int or float, default = 1
        The dataset rescaling factor. A parameter used for rescaling the dataset. 
    encoding : str, default = 'amplit'
        The encoding method used to encode vectors into quantum densities. Possible values:
        'amplit', 'stereo'. 'amplit' means using the amplitude encoding method. 'stereo' means 
        using the inverse of the standard stereographic projection encoding method. Default set 
        to 'amplit'.
    n_copies : int, default = 1
        The number of copies to take for each quantum density. This is equivalent to taking 
        the n-fold Kronecker tensor product for each quantum density.
    measure : str, default = 'pgm'
        The measurement used to distinguish between quantum states. Possible values: 'pgm', 
        'hels'. The value 'pgm' stands for "Pretty Good Measurement", 'hels' stands for 
        "Helstrom measurement" (applicable only for binary classification). Default set to 
        'pgm'.
    class_wgt : str, default = None        
        Applicable only when "Helstrom measurement" is selected. This is the class weights 
        assigned to the Quantum Helstrom observable terms. Possible values: 'equi', 'weighted', 
        None. 'equi' means assigning equal weights of 1/2 (equiprobable) to the two terms in 
        the Quantum Helstrom observable. 'weighted' means assigning weights equal to the 
        proportion of the number of rows in each class to the two terms in the Quantum Helstrom 
        observable. When using "Pretty Good Measurement", specify class_wgt = None. Default set 
        to None.
    n_jobs : int, default = None
        The number of CPU cores used when parallelizing. If -1 all CPUs are used. If 1 is given, 
        no parallel computing code is used at all. For n_jobs below -1, (n_cpus + 1 + n_jobs) 
        are used. Thus for n_jobs = -2, all CPUs but one are used. None is a marker for ‘unset’ 
        that will be interpreted as n_jobs = 1.
    n_splits : int, default = 1
        The number of subset splits performed on the input dataset row-wise and on the number 
        of eigenvalues/eigenvectors of the Quantum Helstrom observable for optimal speed 
        performance. If 1 is given, no splits are performed. For optimal speed, recommend using 
        n_splits = int(numpy.ceil(number of CPU cores used/2)). If memory blow-out occurs, 
        reduce n_splits.
    
    Attributes
    ----------
    classes_ : ndarray, shape (n_classes,)
        Sorted classes.
    qcentroids_ : ndarray, shape (n_classes, (n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Quantum Centroids for each class.
    pgms_ : list, shape (n_classes, (n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Pretty good measurement.
    hels_obs_ : ndarray, shape ((n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Quantum Helstrom observable.
    proj_sums_ : list, shape (2, (n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Sum of the projectors of the Quantum Helstrom observable's unit eigenvectors, which has
        corresponding positive and negative eigenvalues respectively.
    hels_bound_ : float
        Helstrom bound is the upper bound of the probability that one can correctly 
        discriminate whether a quantum density is of which of the two binary quantum density 
        pattern.          
    """
    # Initialize model hyperparameters
    def __init__(self, 
                 rescale = 1,
                 encoding = 'amplit',
                 n_copies = 1, 
                 measure = 'pgm',
                 class_wgt = None, 
                 n_jobs = None, 
                 n_splits = 1):
        self.rescale = rescale
        self.encoding = encoding
        self.n_copies = n_copies
        self.measure = measure
        self.class_wgt = class_wgt
        self.n_jobs = n_jobs
        self.n_splits = n_splits
        
    
    # Function for kronecker tensor product with broadcasting, set as global function
    global kronecker
    def kronecker(A, B):
        return np.einsum('nab,ncd->nacbd', A, B).reshape(A.shape[0],
                                                         A.shape[1]*B.shape[1],
                                                         A.shape[2]*B.shape[2])
    
    
    # Function for fit
    def fit(self, X, y):
        """Perform PGM-HQC classification with the amplitude and inverse of the standard 
        stereographic projection encoding methods, with the option to rescale the dataset prior 
        to encoding.
                
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples. An array of int or float.
        y : array-like, shape (n_samples,)
            The training input binary target values. An array of str, int or float.
            
        Returns
        -------
        self : object
            Returns self.
        """
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        
        # Ensure target y is of non-regression type  
        # Added as required by sklearn check_estimator
        check_classification_targets(y)
    
        # Store classes and encode y into binary class indexes 0 and 1
        self.classes_, y_class_index = np.unique(y, return_inverse = True)
                
        # Number of classes, set as global variable
        global num_classes
        num_classes = len(self.classes_)
        
        # Raise error when there are more than 2 classes and Helstrom measurement is specified
        if num_classes > 2 and self.measure == 'hels':
            raise ValueError('Helstrom measurement can be applied for binary classification only')
        else:
            # Cast X to float to ensure all following calculations below are done in float  
            # rather than integer
            X = X.astype(float)
        
            # Rescale X
            X = self.rescale*X
        
            # Calculate sum of squares of each row (sample) in X
            X_sq_sum = (X**2).sum(axis = 1)
        
            # Number of rows in X
            m = X.shape[0]
        
            # Number of columns in X
            n = X.shape[1]
        
            # Calculate X' using amplitude or inverse of the standard stereographic projection 
            # encoding method
            if self.encoding == 'amplit':
                X_prime = normalize(np.concatenate((X, np.ones(m).reshape(-1, 1)), axis = 1))
            elif self.encoding == 'stereo':
                X_prime = (1 / (X_sq_sum + 1)).reshape(-1, 1) \
                          *(np.concatenate((2*X, (X_sq_sum - 1).reshape(-1, 1)), axis = 1))
            else:
                raise ValueError('encoding should be "amplit" or "stereo"')
        
            # Number of columns in X', set as global variable
            global n_prime
            n_prime = n + 1
        
            # Function to calculate number of rows (samples) and Quantum Centroids for each class
            def qcentroids_terms_func(i):
                # Determine rows (samples) in X' belonging to each class
                X_prime_class = X_prime[y_class_index == i]
            
                # Number of rows (samples) in X' belonging to either class
                m_class = X_prime_class.shape[0]
            
                # Split X' belonging to each class into n_splits subsets, row-wise
                X_prime_class_split = np.array_split(X_prime_class,
                                                     indices_or_sections = self.n_splits,
                                                     axis = 0)
            
                # Function to calculate the Quantum Centroids for each class, per subset split
                def X_prime_class_split_func(j):
                    # Counter for j-th split of X'
                    X_prime_class_split_jth = X_prime_class_split[j]
                
                    # Number of rows (samples) in j-th split of X'
                    m_class_split = X_prime_class_split_jth.shape[0]
                
                    # Encode vectors into quantum densities
                    density_split = np.matmul(X_prime_class_split_jth.reshape(m_class_split, 
                                                                              n_prime, 1),
                                              X_prime_class_split_jth.reshape(m_class_split, 
                                                                              1, n_prime))
                                        
                    # Calculate n-fold Kronecker tensor product
                    if self.n_copies == 1:
                        density_split = density_split
                    else:
                        density_split_copy = density_split
                        for u in range(self.n_copies - 1):
                            density_split = kronecker(density_split, density_split_copy)
                
                    # Calculate sum of quantum densities
                    density_split_sum = density_split.sum(axis = 0)
                
                    # Calculate Quantum Centroid for each class, per subset split
                    # Added ZeroDivisionError as required by sklearn check_estimator
                    try:
                        qcentroid = (1 / m_class)*density_split_sum
                    except ZeroDivisionError:
                        qcentroid = 0
                    return qcentroid
                return m_class, np.sum(Parallel(n_jobs = self.n_jobs)(delayed(X_prime_class_split_func)(j) 
                                                                      for j in range(self.n_splits)), axis = 0)
        
            # Calculate number of rows (samples) and Quantum Centroids for each class
            # Added dtype = object as required by NumPy 19.0 when creating ndarray from ragged nested sequences
            qcentroids_terms = np.array(Parallel(n_jobs = self.n_jobs)(delayed(qcentroids_terms_func)(i) 
                                                                       for i in range(num_classes)), dtype = object)
        
            # Determine Quantum Centroids
            self.qcentroids_ = qcentroids_terms[:, 1]
        
            # When Pretty Good Measurement is specified
            if self.measure == 'pgm':
                if self.class_wgt == None:
                    # Calculate R
                    R = np.sum(self.qcentroids_, axis = 0)
            
                    # Calculate square root of the pseudo_inverse of R
                    sqrt_pinv_R = linalg.sqrtm(np.linalg.pinv(R, hermitian = True))
            
                    # Function to calculate Pretty Good Measurement
                    def pgm_func(a):
                        # Calculate kernel of R
                        ker_R = linalg.null_space(R)
                
                        # Calculate projector of kernel of R
                        proj_ker_R = np.dot(ker_R, ker_R.T)
                        return np.linalg.multi_dot([sqrt_pinv_R, self.qcentroids_[a], sqrt_pinv_R]) \
                               + (1/num_classes)*proj_ker_R
            
                    # Calculate Pretty Good Measurement
                    self.pgms_ = Parallel(n_jobs = self.n_jobs)(delayed(pgm_func)(a) 
                                                                for a in range(num_classes))
                else:
                    raise ValueError('when using "pgm" measure, class_wgt should be None')
            # When Helstrom measurement is specified
            elif self.measure == 'hels':
                # Calculate quantum Helstrom observable
                if self.class_wgt == 'equi':
                    self.hels_obs_ = 0.5*(self.qcentroids_[0] - self.qcentroids_[1])
                elif self.class_wgt == 'weighted':
                    self.hels_obs_ = (qcentroids_terms[0, 0]/m)*self.qcentroids_[0] \
                                     - (qcentroids_terms[1, 0]/m)*self.qcentroids_[1]
                else:
                    raise ValueError('when using "hels" measure, class_wgt should be "equi" or "weighted"')
            
                # Number of rows/columns in density matrix, set as global variable
                global density_nrow_ncol
                density_nrow_ncol = self.hels_obs_.shape[0]
            
                # Calculate eigenvalues w and unit eigenvectors v of the quantum Helstrom observable
                w, v = np.linalg.eigh(self.hels_obs_)
                
                # Length of w
                len_w = len(w)
                
                # Initialize array eigval_class
                eigval_class = np.empty_like(w)
                for b in range(len_w):
                    # Create an array of 0s and 1s to indicate positive and negative eigenvalues
                    # respectively
                    if w[b] > 0:
                        eigval_class[b] = 0
                    else:
                        eigval_class[b] = 1
                        
                # Transpose matrix v containing unit eigenvectors to row-wise
                eigvec = v.T
                
                # Function to calculate sum of the projectors corresponding to positive and negative
                # eigenvalues respectively
                def sum_proj_func(c):
                    # Determine unit eigenvectors belonging to positive and negative eigenvalues 
                    # respectively
                    eigvec_class = eigvec[eigval_class == c]
                    
                    # Split unit eigenvectors into n_splits subsets
                    eigvec_class_split = np.array_split(eigvec_class,
                                                        indices_or_sections = self.n_splits,
                                                        axis = 0)
                    
                    # Function to calculate sum of the projectors corresponding to positive and negative
                    # eigenvalues respectively, per subset split
                    def eigvec_class_split_func(d):
                        # Counter for d-th split of eigvec_class_split
                        eigvec_class_split_dth = eigvec_class_split[d]
                    
                        # Number of rows (samples) in d-th split of eigvec_class_split
                        m_eigvec_class_split = eigvec_class_split_dth.shape[0]
                    
                        # Calculate projectors corresponding to positive and negative eigenvalues
                        # respectively, per subset split
                        proj_split = np.matmul(eigvec_class_split_dth.reshape(m_eigvec_class_split,
                                                                              density_nrow_ncol, 1),
                                               eigvec_class_split_dth.reshape(m_eigvec_class_split,
                                                                              1, density_nrow_ncol))

                        # Calculate sum of projectors
                        proj_split_sum = proj_split.sum(axis = 0)
                        return proj_split_sum
                    return np.sum(Parallel(n_jobs = self.n_jobs)(delayed(eigvec_class_split_func)(d) 
                                                                 for d in range(self.n_splits)), axis = 0)
                
                # Calculate sum of the projectors corresponding to positive and negative eigenvalues
                # respectively
                self.proj_sums_ = Parallel(n_jobs = self.n_jobs)(delayed(sum_proj_func)(c) 
                                                                 for c in range(num_classes))
                
                # Calculate Helstrom bound
                self.hels_bound_ = (qcentroids_terms[0, 0] / m)*np.einsum('ij,ji->', self.qcentroids_[0],                                                                      
                                                                          self.proj_sums_[0]) \
                                   + (qcentroids_terms[1, 0] / m)*np.einsum('ij,ji->', self.qcentroids_[1],
                                                                            self.proj_sums_[1])
            # When Pretty Good Measurement or Helstrom measurement is misspecified
            else:
                raise ValueError('measure should be "pgm" or "hels"')
            return self             

    
    # Function for predict_proba
    def predict_proba(self, X):
        """Performs PMG-HQC classification on X and returns the trace of the dot product of the 
        densities and the POV (positive operator-valued) measure, i.e. the class probabilities.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.       
            
        Returns
        -------
        trace_matrix : ndarray, shape (n_samples, n_classes)
            Each column corresponds to the trace of the dot product of the densities and the POV 
            (positive operator-valued) measure for each class, i.e. each column corresponds to the 
            class probabilities. An array of float.
        """
        # Check if fit had been called
        if self.measure == 'pgm':
            check_is_fitted(self, ['pgms_'])
        else:
            check_is_fitted(self, ['proj_sums_'])

        # Input validation
        X = check_array(X)
        
        # Cast X to float to ensure all following calculations below are done in float 
        # rather than integer
        X = X.astype(float)        
        
        # Rescale X
        X = self.rescale*X        
        
        # Calculate sum of squares of each row (sample) in X
        X_sq_sum = (X**2).sum(axis = 1)
        
        # Number of rows in X
        m = X.shape[0]
        
        # Number of columns in X
        n = X.shape[1]

        # Calculate X' using amplitude or inverse of the standard stereographic projection 
        # encoding method
        if self.encoding == 'amplit':
            X_prime = normalize(np.concatenate((X, np.ones(m).reshape(-1, 1)), axis = 1))
        elif self.encoding == 'stereo':
            X_prime = (1 / (X_sq_sum + 1)).reshape(-1, 1) \
                      *(np.concatenate((2*X, (X_sq_sum - 1).reshape(-1, 1)), axis = 1))
        else:
            raise ValueError('encoding should be "amplit" or "stereo"')
               
        # Function to calculate trace values for each class
        def trace_func(i):
            # Split X' into n_splits subsets, row-wise
            X_prime_split = np.array_split(X_prime, 
                                           indices_or_sections = self.n_splits, 
                                           axis = 0)
            
            # Function to calculate trace values for each class, per subset split
            def trace_split_func(j):
                # Counter for j-th split X'
                X_prime_split_jth = X_prime_split[j]
                
                # Number of rows (samples) in j-th split X'
                X_prime_split_m = X_prime_split_jth.shape[0]
                
                # Encode vectors into quantum densities
                density_chunk = np.matmul(X_prime_split_jth.reshape(X_prime_split_m, n_prime, 1),
                                          X_prime_split_jth.reshape(X_prime_split_m, 1, n_prime))
                
                # Calculate n-fold Kronecker tensor product
                if self.n_copies == 1:
                    density_chunk = density_chunk
                else:
                    density_chunk_copy = density_chunk
                    for u in range(self.n_copies - 1):
                        density_chunk = kronecker(density_chunk, density_chunk_copy)
                        
                # When Pretty Good Measurement is specified
                if self.measure == 'pgm':
                    # Calculate trace of the dot product of density of each row and Pretty Good 
                    # Measurement
                    trace_class_split = np.einsum('bij,ji->b', density_chunk, self.pgms_[i])
                # When Helstrom measurement is specified
                else:               
                    # Calculate trace of the dot product of density of each row and sum of 
                    # projectors with corresponding positive and negative eigenvalues respectively
                    trace_class_split = np.einsum('bij,ji->b', density_chunk, self.proj_sums_[i])
                return trace_class_split
            
            # Calculate trace values for each class, per subset split
            trace_class = Parallel(n_jobs = self.n_jobs)(delayed(trace_split_func)(j) 
                                                         for j in range(self.n_splits))
            return np.concatenate(trace_class, axis = 0)
        
        # Calculate trace values for each class
        trace_matrix = np.transpose(Parallel(n_jobs = self.n_jobs)(delayed(trace_func)(i) 
                                                                   for i in range(num_classes)))
        return trace_matrix
        
    
    # Function for predict
    def predict(self, X):
        """Performs PGM-HQC classification on X and returns the classes.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.
            
        Returns
        -------
        self.classes_[predict_trace_index] : ndarray, shape (n_samples,)
            The predicted classes. An array of str, int or float.
        """
        # Determine column index with the higher trace value in trace_matrix
        # If columns have the same trace value, returns column with the smallest column index value
        predict_trace_index = np.argmax(self.predict_proba(X), axis = 1)
        # Returns the predicted classes
        return self.classes_[predict_trace_index]

In [16]:
# new-balance dataset (5 features, 625 rows)
import pandas as pd

df = pd.read_csv('balance-scale.tsv', sep='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

In [17]:
# Check if class imbalance
df['target'].value_counts()

2    288
1    288
0     49
Name: target, dtype: int64

In [42]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

# Check F1 score for various n_copies values
model = PGMHQC_fast(rescale=1, n_copies=2, encoding='stereo', measure='pgm', class_wgt=None, n_jobs=16, n_splits=8).fit(X_train, y_train)

# Check trace values sum to 1 for first 5 rows
np.concatenate([model.predict_proba(X_test), np.sum(model.predict_proba(X_test), axis=1).reshape(-1,1)], axis=1)[0:5,:]

array([[0.31456559+1.26704238e-24j, 0.32840785+1.14129599e-24j,
        0.35702656+1.30101402e-24j, 1.        +3.70935239e-24j],
       [0.31538978+1.52092325e-24j, 0.32799515+1.33201361e-24j,
        0.35661507+1.64182749e-24j, 1.        +4.49476435e-24j],
       [0.31522483+7.97200774e-25j, 0.34288295+6.46049450e-25j,
        0.34189222+9.12249769e-25j, 1.        +2.35549999e-24j],
       [0.32294977+8.84068674e-25j, 0.31626982+4.43697165e-25j,
        0.3607804 +1.12133980e-24j, 1.        +2.44910563e-24j],
       [0.31668389+4.34293294e-25j, 0.34181528+2.65984014e-25j,
        0.34150083+5.96786218e-25j, 1.        +1.29706353e-24j]])

In [43]:
# Check F1 score for various n_copies values
model = PGMHQC_fast(rescale=1, n_copies=1, encoding='stereo', measure='pgm', class_wgt=None, n_jobs=16, n_splits=8).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8367315008833559

In [44]:
# Check F1 score for various n_copies values
model = PGMHQC_fast(rescale=1, n_copies=2, encoding='stereo', measure='pgm', class_wgt=None, n_jobs=16, n_splits=8).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8447445962196664

In [45]:
# Check F1 score for various n_copies values
model = PGMHQC_fast(rescale=1, n_copies=3, encoding='stereo', measure='pgm', class_wgt=None, n_jobs=16, n_splits=8).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8605017010473518

In [46]:
# Check F1 score for various n_copies values
model = PGMHQC_fast(rescale=1, n_copies=4, encoding='stereo', measure='pgm', class_wgt=None, n_jobs=16, n_splits=8).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8682253173918403

In [47]:
# Check F1 score for various n_copies values
model = PGMHQC_fast(rescale=1, n_copies=5, encoding='stereo', measure='pgm', class_wgt=None, n_jobs=16, n_splits=8).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8678604178430266

In [48]:
# Check F1 score for various n_copies values
model = PGMHQC_fast(rescale=1, n_copies=6, encoding='stereo', measure='pgm', class_wgt=None, n_jobs=16, n_splits=8).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

MemoryError: Unable to allocate 52.8 GiB for an array with shape (29, 3125, 5, 3125, 5) and data type float64

In [39]:
# Testing using scikit-learn's GridSearchCV
from sklearn.model_selection import GridSearchCV

# Create rescale hyperparamter list [0.1, 0.5, 1, 1.5,...,10.0]
rescale_list1 = [0.1]
rescale_list2 = np.linspace(0.5, 10, 20).tolist()
rescale_list1.extend(rescale_list2)

param_grid = {'rescale':rescale_list1, 'n_copies':[1, 2, 3, 4], 'encoding':['amplit', 'stereo'], 'measure':['pgm'], 'class_wgt':[None]}
models = GridSearchCV(PGMHQC_fast(n_jobs=16, n_splits=8), param_grid, scoring='f1_weighted').fit(X_train, y_train)

In [40]:
# Best F1 score
best_model = models.best_estimator_
y_hat = best_model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.9106013291733841

In [41]:
# Best hyperparameter combination
models.best_params_

{'class_wgt': None,
 'encoding': 'amplit',
 'measure': 'pgm',
 'n_copies': 2,
 'rescale': 2.0}